# Projeto

Utilizando o dataset de revisões de filmes em português (baixado com ajuda do código abaixo), criar um classificador de sentimentos que consiga um score na métrica **F1 Score** superior a 70%.

Fique a vontade para testar métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.

O trabalho poderá ser feito em grupo (mesmos grupos do Startup One).

A data de entrega é dia 08/02/2019 até as 23:59 horas,


In [37]:
import pandas as pd

df = pd.read_csv("https://storage.googleapis.com/buck-12/imdb-reviews-pt-br.csv", delimiter=",", encoding='utf-8')

df.describe()

,id
count,49459.000000
mean,24730.960917
std,14277.792868
min,1.000000
25%,12366.500000
50%,24731.000000
75%,37095.500000
max,49460.000000


In [23]:
df.head(5)

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


Embaralhamos os dados para tentar garantir que exista a mesma quantidade de avaliações positivas e negarivas para treino e teste. Em seguida ajustamos para que positivo seja considerado 1 e negativo 0 e removemos as stopwords dos textos

In [45]:
import nltk

df = df.sample(frac=1).reset_index(drop=True)
df['sentiment'] = df['sentiment'].map({'pos': 1, 'neg': 0, 1: 1, 0: 0})

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

def removerStopwords(frase):
  lista_frase = frase.split()
  result = [palavra for palavra in lista_frase if palavra.lower() not in stopwords]
  return ' '.join(result)

df.text_pt = df.text_pt.apply(removerStopwords)

df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text_en,text_pt,sentiment
0,18555,For those of you who like stand-up comedians y...,"gostam comediantes, deve ter ouvido falar Geor...",1
1,13835,The film is partly a thriller and partly a pub...,filme é parte thriller parte anúncio serviço p...,1
2,30527,"Im a true fan of the original Cracker series, ...",verdadeiro fã série original Cracker possuo to...,0
3,31039,Dane tries to hard and is to extreme with all ...,Dane tenta esforçar vai extremo todos gritos e...,0
4,634,This movie is funny if youre the gentleman who...,filme é engraçado é cavalheiro sentado três fi...,0


Separamos os dados para treino e teste e utilizamos 30% para teste

In [0]:
from sklearn.model_selection import train_test_split

X = df.text_pt
y = df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Treinamos um modelo linear SVC para realizar a analise de sentimento. Fizemos os testes utilizando vetorização com unigrama, bigrama e trigrama

In [53]:
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer


# Vetoriza os textos
vect = CountVectorizer(ngram_range=(1,1))

training_features = vect.fit_transform(X_train)    
test_features = vect.transform(X_test)

# Treina um modelo SVC
modelUni = LinearSVC()
modelUni.fit(training_features, y_train)
y_predUni = modelUni.predict(test_features)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [54]:
# Vetoriza os textos
vect = CountVectorizer(ngram_range=(1,2))

training_features = vect.fit_transform(X_train)    
test_features = vect.transform(X_test)

# Treina um modelo SVC
modelBi = LinearSVC()
modelBi.fit(training_features, y_train)
y_predBi = modelBi.predict(test_features)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [55]:
# Vetoriza os textos
vect = CountVectorizer(ngram_range=(1,3))

training_features = vect.fit_transform(X_train)    
test_features = vect.transform(X_test)

# Treina um modelo SVC
modelTri = LinearSVC()
modelTri.fit(training_features, y_train)
y_predTri = modelTri.predict(test_features)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [56]:
# F1 Score

f1Uni = f1_score(y_test, y_predUni)
f1Bi = f1_score(y_test, y_predBi)
f1Tri = f1_score(y_test, y_predTri)
print("F1 Score com unigrama: {:.2f}".format(f1Uni))
print("F1 Score com bigrama: {:.2f}".format(f1Bi))
print("F1 Score com trigrama: {:.2f}".format(f1Tri))

F1 Score com unigrama: 0.86
F1 Score com bigrama: 0.89
F1 Score com trigrama: 0.89
